In [67]:
import cv2
#import pandas as pd

import numpy as np
# 2cm x 2cm


In [68]:
not_detect_imgs = []
corners_dict = {}
for idx in range(1,26): # testing first two imgs
    img = cv2.imread(f"img{idx}.jpg")
    img = np.uint8(img)
    gray_scaled_img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)

    ans, corners= cv2.findChessboardCorners(gray_scaled_img, patternSize= (9,6))
    if ans is False:
        not_detect_imgs.append(idx)
    else:
        corners_dict[idx] = corners
    #print(len(corners))
    
    
    #print(corners)
    # x = corners[0][0]
    # y = corners[0][1]
    # print(f"x: {x}, y: {y}")
    # print("ans", ans)
    # print('corner:', corners)
    # print('------------')

# print(not_detect_imgs)
# print(corners_dict)

print(not_detect_imgs)

[1, 4, 5, 6, 10, 12, 13, 19, 20]


In [69]:

for corners in corners_dict.values():
    for coords in corners: #2621. 2016.5
        #print(i[0])
        x = coords[0][0]
        y = coords[0][1]
        print(x,y)
        center = (int(x), int(y))
        print(center)
        break

544.97595 316.93356
(544, 316)
610.4357 336.41296
(610, 336)
602.717 356.7525
(602, 356)
627.7335 378.20477
(627, 378)
608.5 427.0
(608, 427)
601.5 171.5
(601, 171)
551.50494 238.26869
(551, 238)
558.17255 157.28372
(558, 157)
704.89825 227.15437
(704, 227)
703.5418 299.07608
(703, 299)
610.5784 289.06122
(610, 289)
551.0498 374.34402
(551, 374)
555.0785 355.52313
(555, 355)
519.7551 392.36423
(519, 392)
637.2728 403.72522
(637, 403)
641.91614 278.5898
(641, 278)


In [70]:
square_size = 0.02 
obj_pts = np.zeros((9*6, 3), np.float32)
obj_pts[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2) * square_size

In [71]:

# square_size = 0.02 #2 cm
# obj_pts = np.zeros((9*6, 3), np.float32)
# obj_pts[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2) * square_size


# all_obj_pts = []
# all_img_pts = []

# auto_only_imgs = []

# manual_only_imgs = []

# def click_event(event, x, y, flags, param):
#     current_img = param['img']
#     current_name = param['name']
#     if event == cv2.EVENT_LBUTTONDBLCLK:
#         print(x,y) # coordinate
#         font = cv2.FONT_HERSHEY_SIMPLEX
#         cv2.putText(current_img, f'{x}, {y}', (x,y), font, 1, (255,0,0), 2)
#         cv2.imshow(window_name, current_img)
#         param['point'].append([x,y])
    
#     if event == cv2.EVENT_RBUTTONDBLCLK:
#         print(x,y)
#         font = cv2.FONT_HERSHEY_SIMPLEX
#         b,g,r = current_img[y,x] 
#         cv2.putText(img,f'{b}, {g}, {r}', (x,y), font, 1, (255, 255,0), 2)
#         cv2.imshow(window_name, current_img)
#         #points += [x,y]

#     print(f"Manual Points: {param['point']}")



# # linear interpolation
# def linear_interpolation(pts, pattern_size):
#     tl, tr, br, bl = np.array(pts[0],dtype = 'float32'),  np.array(pts[1],dtype = 'float32'), np.array(pts[2],dtype = 'float32'), np.array(pts[3],dtype = 'float32')
#     #print(type(tl))
#     cols, rows = pattern_size
#     print(cols)
#     grid = []
#     for row in range(rows):
#         ver_int = row / (rows - 1)
#         row_start = (1 - ver_int) * tl + ver_int * bl
#         row_end = (1 - ver_int)*  tr + ver_int * br
        
#         for col in range(cols):
#             hor_int = col / (cols -1)
#             point = (1 - hor_int) * row_start + hor_int * row_end
#             grid.append(point)
#     return np.array(grid, dtype= 'float32').reshape(-1, 1, 2)

# pattern_size = (9,6)

# window_name = 'chessboard_tool'
# all_result = {}
# # read input image
# for idx in range(1,26):

#     img = cv2.imread(f'img{idx}.jpg')

# # convert the input image to a grayscale
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#     # Find the chess board corners
#     ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
#     print(corners)
#     # if chessboard corners are detected
#     if ret == True:
    
#         # Draw and display the corners
#         img = cv2.drawChessboardCorners(img, (9,6), corners,ret)
#         all_img_pts.append(corners)
#         auto_only_imgs.append(corners)
#         all_obj_pts.append(obj_pts)

#         cv2.imshow(window_name,img)
#         cv2.waitKey(0)

#     elif ret == False:
#         # clicking (TL -> TR -> BR -> BL)
#         manual_pts = []
#         cv2.imshow(window_name, img)
#         cv2.setMouseCallback(window_name, click_event, param= {'img': img, 'name': f'Image{idx},', 'point':manual_pts})
#         cv2.waitKey(0)



#         if len(manual_pts) == 4:
#             interpolated_corners = linear_interpolation(manual_pts, (9,6))
#             all_img_pts.append(interpolated_corners)
#             manual_only_imgs.append(interpolated_corners)
#             all_obj_pts.append(obj_pts)

#             img_vis = img.copy()
#             cv2.drawChessboardCorners(img_vis, (9,6), interpolated_corners, True)
#             cv2.imshow(window_name, img_vis)
#             print("Showing interpolated grid. Press any key to continue to next image.")
#             cv2.waitKey(0) # Wait again so you can actually see the result
        
            
        
#         else:
#             print(f"Warning: Only {len(manual_pts)} points clicked. Skipping image {idx}.")


#     cv2.destroyWindow(window_name)



In [72]:
#print(manual_only_imgs)
import joblib

# obj_points = joblib.dump(all_obj_pts , 'obj_points.joblib')
# img_points = joblib.dump(all_img_pts, 'img_points.joblib')

# auto_points = joblib.dump(auto_only_imgs, 'auto_points.joblib')
# manual_points = joblib.dump(manual_only_imgs, 'manual_points.joblib')

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
all_obj_pts = joblib.load('obj_points.joblib')
all_img_pts = joblib.load('img_points.joblib')
auto_only_imgs = joblib.load('auto_points.joblib')
manual_only_imgs = joblib.load('manual_points.joblib')


def calibrate(object_list, img_list, gray_img):

    ret, matrix, distort, rvecs, tvec = cv2.calibrateCamera(object_list, img_list, gray_img, None, None, flags = 0)

    return ret, matrix, distort, rvecs, tvec

#Run 1: All Images

ret1, matrix1, distort1, rvecs1, tvec1 = calibrate(all_obj_pts, all_img_pts, gray.shape[::-1] )

#Run 2: Only Five images each
run2_imgs = auto_only_imgs[0:5] + manual_only_imgs[0:5]
print(len(run2_imgs))
object_10 = [obj_pts]*10

ret2, matrix2, distort2, rvecs2, tvec2 = calibrate(object_10, run2_imgs, gray.shape[::-1] )

#Run 3: Automatic 5 images

run3_imgs = auto_only_imgs[:5] 

object_5_3 = [obj_pts]*5

ret3, matrix3, distort3, rvecs3, tvec3 = calibrate(object_5_3, run3_imgs, gray.shape[::-1] )


10


In [73]:
test_img = cv2.imread('test.jpg')
test_gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

ret_test, test_corners = cv2.findChessboardCorners(test_gray, (9,6), None)


ret, rvec, tvec= cv2.solvePnP(obj_pts, test_corners, matrix1, distort1)

In [74]:
#creating the axis here
axis = np.float32([[0.06, 0 ,0], [0, 0.06, 0], [0, 0,-0.06]]).reshape(-1,3)

#cube: 8 corners

s = 0.04

cube_points = np.float32([[0,0,0], [s,0,0], [s,s,0], [0,s,0], [0,0,-s], [s,0,-s], [s,s,-s], [0,s,-s]])


In [75]:
#We want to project the axis onto image
imgpoints_axis, _ = cv2.projectPoints(axis, rvec, tvec, matrix1, distort1)

imgpoints_cube, _ = cv2.projectPoints(cube_points, rvec, tvec, matrix1, distort1)




In [76]:
print(imgpoints_cube)

[[[637.86127 370.38126]]

 [[630.0841  435.7    ]]

 [[553.5244  435.0731 ]]

 [[559.72516 369.45975]]

 [[642.611   470.6378 ]]

 [[634.6596  535.1825 ]]

 [[556.7797  534.91846]]

 [[563.25903 470.08252]]]


In [77]:
origin_pixel = tuple(test_corners[0].ravel().astype(int))
print(origin_pixel)

(639, 369)


In [78]:
# Convert your projected points to integers for drawing
axis_pix = imgpoints_axis.reshape(-1, 2).astype(int)
cube_pix = imgpoints_cube.reshape(-1, 2).astype(int)
# Draw X axis (Red)
cv2.line(test_img, origin_pixel, tuple(axis_pix[0]), (0, 0, 255 ), 5)
cv2.line(test_img, origin_pixel, tuple(axis_pix[1]), (0, 255, 0), 5)
cv2.line(test_img, origin_pixel, tuple(axis_pix[2]), (255, 0, 0), 5)

cv2.line(test_img, tuple(cube_pix[0]), tuple(cube_pix[4]), (0, 0, 0), 2)
cv2.line(test_img, tuple(cube_pix[1]), tuple(cube_pix[5]), (0, 0, 0), 2)
cv2.line(test_img, tuple(cube_pix[2]), tuple(cube_pix[6]), (0, 0, 0), 2)
cv2.line(test_img, tuple(cube_pix[3]), tuple(cube_pix[7]), (0, 0, 0), 2)

cv2.polylines(test_img, [cube_pix[0:4]], True, (0, 0, 0), 2)
cv2.polylines(test_img, [cube_pix[4:8]], True, (0, 0, 0), 2)


cv2.imshow('chessboard_window', test_img)
cv2.waitKey(0)
cv2.destroyWindow('chessboard_window')